In [1]:
import random
from collections import defaultdict
from itertools import combinations
from tabulate import tabulate

## BASE DICE ROLL

In [9]:
uppersection = ["Ones", "Twos", "Threes", "Fours", "Fives", "Sixes"]

In [10]:
yahtzeecategories = uppersection + [
    "Three-of-a-Kind", "Four-of-a-Kind", "Full House",
    "Small Straight", "Large Straight", "Lower Section (Yahtzee)", "Chance"
]

In [11]:
def roll_dice(keep=[]): #dice roll
    return sorted(keep + [random.randint(1, 6) for _ in range(5 - len(keep))])

In [12]:
def score_hand(dice, category): #scoring
    counts = {x: dice.count(x) for x in set(dice)}
    if category in uppersection:
        face = uppersection.index(category) + 1
        return dice.count(face) * face
    if category == "Three-of-a-Kind" and any(v >= 3 for v in counts.values()):
        return sum(dice)
    if category == "Four-of-a-Kind" and any(v >= 4 for v in counts.values()):
        return sum(dice)
    if category == "Full House" and sorted(counts.values()) == [2, 3]:
        return 25
    if category == "Small Straight" and any(all(i in dice for i in seq) for seq in ([1,2,3,4],[2,3,4,5],[3,4,5,6])):
        return 30
    if category == "Large Straight" and sorted(dice) in ([1,2,3,4,5],[2,3,4,5,6]):
        return 40
    if category == "Lower Section (Yahtzee)" and any(v == 5 for v in counts.values()):
        return 50
    if category == "Chance":
        return sum(dice)
    return 0


In [13]:
def best_move(dice, category):
    counts = {x: dice.count(x) for x in set(dice)}
    if category in uppersection:
        face = uppersection.index(category) + 1
        return [d for d in dice if d == face]
    if category == "Three-of-a-Kind" or category == "Four-of-a-Kind" or category == "Lower Section (Yahtzee)":
        target = max(counts, key=counts.get)
        return [d for d in dice if d == target]
    if category == "Large Straight":
        return list(set(dice) & {2,3,4,5})
    if category == "SixofaKind":
        return [d for d in dice if d == 6]
    return dice

## UPPER SECTION OPTIMIZATION

In [15]:
def simulate_full_game(category_usage): #function to simulate the full game using the dice, best move and score hand functions created in the previous section of this code
    remaining = yahtzeecategories.copy() #copy of the remaining category function created in the previous section
    total_score = 0 #tracks the total score on all game rounds
    scorecard = {} #blank score card to keep track of each play and what category was played saved as sets
    upper_total = 0 #tracks the total points in the upper section

    for _ in range(len(yahtzeecategories)):
        dice = roll_dice() #inital roll
        for _ in range(2): #opportunity for a player to reroll twice based on the remaining category
            best_score = -1
            best_keep = []
            best_cat = None
            for cat in remaining:
                keep = best_move(dice, cat)
                test_dice = roll_dice(keep)
                s = score_hand(test_dice, cat)
                if s > best_score: #keeps the best hand
                    best_score = s
                    best_keep = keep
                    best_cat = cat
            dice = roll_dice(best_keep)

        best_final_score = -1
        best_final_cat = None
        for cat in remaining: #basically keeping the the hightes score out of all rolls
            s = score_hand(dice, cat)
            if s > best_final_score:
                best_final_score = s
                best_final_cat = cat

        scorecard[best_final_cat] = best_final_score
        category_usage[best_final_cat]["count"] += 1
        category_usage[best_final_cat]["total"] += best_final_score

        if best_final_cat in uppersection:
            upper_total += best_final_score
        total_score += best_final_score #add total score and remove any unused category
        remaining.remove(best_final_cat)

    if upper_total >= 63: #bonus section score awarded if the rolls meet the bonus threhold
        total_score += 35
        scorecard['Upper Bonus'] = 35
        category_usage['Upper Bonus']["count"] += 1
        category_usage['Upper Bonus']["total"] += 35
    else:
        scorecard['Upper Bonus'] = 0
        category_usage['Upper Bonus']["count"] += 1
        category_usage['Upper Bonus']["total"] += 0

    return total_score, scorecard


def simulate_multiple_games(n): #simulate multiple games
    scores = []
    category_usage = defaultdict(lambda: {"count": 0, "total": 0})

    for y in range(n):
        total, y = simulate_full_game(category_usage)
        scores.append(total)

    avg = sum(scores) / n
    best_score = max(scores)
    worst_score = min(scores)

    print(f"{n} simulations")
    print(f"Average score: {avg:.2f}")
    print(f"Best score: {best_score} & Worst score: {worst_score}")

    print("Summary")
    summary = []
    for category, stats in category_usage.items():
        count = stats["count"]
        avg_score = stats["total"] / count if count > 0 else 0
        summary.append((category, count, round(avg_score, 2)))

    summary.sort()
    print(tabulate(summary, headers=["Category", "Number of times used", "Average score"]))
    return scores


if __name__ == "__main__": #this code runs the game for the specified number of turns, I have specified 100000 games here
    simulate_multiple_games(100000)

100000 simulations
Average score: 176.08
Best score: 337 & Worst score: 54
Summary
Category                   Number of times used    Average score
-----------------------  ----------------------  ---------------
Chance                                   100000            18.29
Fives                                    100000            10.89
Four-of-a-Kind                           100000            13.14
Fours                                    100000             9.06
Full House                               100000            19.24
Large Straight                           100000            16.85
Lower Section (Yahtzee)                  100000            13.22
Ones                                     100000             1.94
Sixes                                    100000            12.85
Small Straight                           100000            28.61
Three-of-a-Kind                          100000            19.88
Threes                                   100000             6.73
Twos   

## LOWER SECTION OPTIMIZATION

In [17]:
def simulate_full_game(category_usage): #function to simulate the full game using the dice, best move and score hand functions created in the previous section of this code
    remaining = yahtzeecategories.copy() #copy of the remaining category function created in the previous section
    total_score = 0 #tracks the total score on all game rounds
    scorecard = {} #blank score card to keep track of each play and what category was played saved as sets
    upper_total = 0 #tracks the total points in the upper section
    yahtzee_bonus = 0 #this is to optimize the lower section so it includes the yahtzee bonus for the lower section

    for _ in range(len(yahtzeecategories)):
        dice = roll_dice()
        for _ in range(2):
            best_score = -1
            best_keep = []
            best_cat = None
            for cat in remaining:
                keep = best_move(dice, cat)
                test_dice = roll_dice(keep)
                s = score_hand(test_dice, cat)
                if s > best_score:
                    best_score = s
                    best_keep = keep
                    best_cat = cat
            dice = roll_dice(best_keep)

        best_final_score = -1
        best_final_cat = None
        for cat in remaining:
            s = score_hand(dice, cat)
            if s > best_final_score:
                best_final_score = s
                best_final_cat = cat

        scorecard[best_final_cat] = best_final_score
        category_usage[best_final_cat]["count"] += 1
        category_usage[best_final_cat]["total"] += best_final_score

        if best_final_cat == "Lower Section (Yahtzee)" and best_final_score == 50:#the main difference between the lower section simulation and the upper section simulation is the addition of teh bonus section score
            yahtzee_bonus += 100

        if best_final_cat in uppersection:
            upper_total += best_final_score
        total_score += best_final_score
        remaining.remove(best_final_cat)

    if upper_total >= 63:
        total_score += 35
        scorecard['Upper Bonus'] = 35
        category_usage['Upper Bonus']["count"] += 1
        category_usage['Upper Bonus']["total"] += 35
    else:
        scorecard['Upper Bonus'] = 0
        category_usage['Upper Bonus']["count"] += 1
        category_usage['Upper Bonus']["total"] += 0

    if yahtzee_bonus > 0:
        total_score += yahtzee_bonus
        scorecard['Yahtzee Bonus'] = yahtzee_bonus
        category_usage['Yahtzee Bonus']["count"] += 1
        category_usage['Yahtzee Bonus']["total"] += yahtzee_bonus

    return total_score, scorecard


# using the same simulate_multiple games code from the previous simulation
if __name__ == "__main__":
    simulate_multiple_games(100000)

100000 simulations
Average score: 202.34
Best score: 426 & Worst score: 59
Summary
Category                   Number of times used    Average score
-----------------------  ----------------------  ---------------
Chance                                   100000            18.3
Fives                                    100000            10.89
Four-of-a-Kind                           100000            13.1
Fours                                    100000             9.04
Full House                               100000            19.23
Large Straight                           100000            16.68
Lower Section (Yahtzee)                  100000            13.23
Ones                                     100000             1.93
Sixes                                    100000            12.85
Small Straight                           100000            28.63
Three-of-a-Kind                          100000            19.9
Threes                                   100000             6.7
Twos       